# Examine spatial patterning of genes around hub+ and hub- interfaces in MSI samples

# load libraries

In [ ]:
require(tidyverse)
require(sf)
require(ggthemes)
require(ggpubr)
require(scattermore)
require(data.table)
require(future)
require(furrr)
require(nngeo)
require(patchwork)
require(mgcv)
require(marginaleffects)
require(Seurat)
require(slider)
require(rstatix)
require(ggpubr)
require(ggnewscale)

In [ ]:
fig.size = function(height, width, res = 300){
    options(repr.plot.width = width, repr.plot.height = height, repr.plot.res = res)
}
fixTheme = ggpubr::theme_pubr(base_family = "Helvetica", base_size = 12) + 
          theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 0.5))
fixGuides = guides(fill = guide_legend(override.aes = list(shape = 16, size = 12)), 
                 color = guide_legend(override.aes = list(shape = 16, size = 12)))
tumor_palette =  c("CXCL+ stroma" = "#1e3888",
                   "CXCL- stroma" = "#47a8bd",
                   "CXCL+ tumor" = "#f5e663",
                   "CXCL- tumor" = "#ffad69",
                   "CXCL+ tumor-stromal interface" = 'red') #"#9c3848"
lineage_palette = c('Epi' = '#CA49FC',
        'Strom' = '#00D2D0',
        'Myeloid' = '#FFB946',
        'Mast' = '#F4ED57',
        'Plasma' = '#61BDFC',
        'B' = '#0022FA',
        'TNKILC' = '#FF3420'
        )

# load data

### Tessera tile metadata and cell-level metadata

In [ ]:
agg_metadata = readr::read_rds('../Tessera tiles/Tessera processed results/agg_metadata_2025-07-22.rds')
tile_metadata = readr::read_rds('../Tessera tiles/Tessera processed results/tile_metadata_2025-07-22.rds')
tile_metadata$type_lvl1[tile_metadata$type_lvl2 == 'Mast'] = 'Mast' 

In [ ]:
unique(tile_metadata$SampleID[tile_metadata$Status == 'MSI'])

In [ ]:
tile_metadata = tile_metadata %>%
    mutate(type_lvl3 = gsub(type_lvl2, pattern = '-prolif|high|low', replacement = '')) %>%
    mutate(type_lvl3 = gsub(type_lvl3, pattern = 'Epi.*', replacement = 'Epi'))

In [ ]:
sample_n(agg_metadata, 10)
dim(agg_metadata)
colnames(agg_metadata)

In [ ]:
sample_n(tile_metadata, 10)
dim(tile_metadata)
colnames(tile_metadata)

### table of distances of each cell to the nearest interface

In [ ]:
# Load necessary libraries
# Ensure you have these installed: install.packages(c("data.table", "Matrix", "spatula", "presto", "furrr", "dplyr", "metafor"))
library(data.table)
library(Matrix)
library(spatula)
library(presto)
library(furrr)
library(dplyr)
library(metafor)

#' Calculate Cell Type Spatial Enrichment
#'
#' This function analyzes spatial relationships between cell types on a per-sample basis.
#' For each "anchor" cell, it counts the number of "neighbor" cells within a defined
#' neighborhood and uses a mixed-effects model (via presto) to determine if certain
#' neighbor types are enriched around certain anchor types.
#'
#' @param cells A data.table or data.frame containing cell data. Must include columns
#'   for 'SampleID', spatial coordinates 'X' and 'Y', and cell type 'type_lvl3'.
#' @param anchor_types A character vector specifying the cell types to be treated as
#'   the central "anchor" cells in the analysis.
#' @param neighbor_types A character vector specifying the cell types to be counted
#'   as "neighbors".
#' @param max_dist A numeric value for the maximum distance (in the same units as X/Y
#'   coordinates) to consider two cells neighbors. Connections longer than this are
#'   pruned. Defaults to 30.
#' @param nsteps An integer specifying the number of steps to expand the neighborhood.
#'   nsteps = 1 means immediate neighbors, nsteps = 2 includes neighbors of neighbors, etc.
#'   Defaults to 3.
#' @param use_multicore A logical value indicating whether to process samples in parallel
#'   using `furrr` with the `multicore` plan. Defaults to TRUE.
#'
#' @return A data.table containing the enrichment analysis results for each sample,
#'   with columns for anchor type, neighbor type, log-fold change, standard error, etc.
#'
calculate_spatial_enrichment <- function(cells,
                                         anchor_types,
                                         neighbor_types,
                                         max_dist = 30,
                                         nsteps = 3,
                                         use_multicore = TRUE) {
  
  # Set up the parallel processing plan based on the user's choice.
  # plan(sequential) runs tasks one by one.
  # plan(multicore) runs tasks in parallel on the same machine.
  if (use_multicore) {
    plan(multicore)
  } else {
    plan(sequential)
  }
  
  # Process each sample independently.
  # We split the main 'cells' data.table by 'SampleID'.
  # 'future_imap' then iterates over each sample's data chunk.
  res_all <- cells %>%
    split(.$SampleID) %>%
    future_imap(function(.cells, .name) {
      
      # Print the name of the sample being processed for progress tracking.
      message("Processing sample: ", .name)
      
      # --- 1. Define Spatial Neighbors ---
      
      # Extract X and Y coordinates into a matrix.
      coords <- as.matrix(.cells[, .(X, Y)])

      # Center the coordinates without changing their scale
      #coords <- scale(coords, center = TRUE, scale = FALSE)
      # 'getSpatialNeighbors' creates an adjacency matrix where non-zero entries
      # indicate neighboring cells and the values are the distances.
      adj <- spatula::getSpatialNeighbors(coords, return_weights = TRUE)
      
      # Prune long-distance connections: set distances greater than 'max_dist' to 0.
      adj@x[adj@x > max_dist] <- 0
      
      # 'drop0' removes the explicit zero entries, making the matrix sparse and efficient.
      adj <- Matrix::drop0(adj)
      
      # Convert the adjacency matrix to be unweighted (binary: 1 for neighbor, 0 otherwise).
      adj@x <- rep(1, length(adj@x))
      
      # Sanity check: ensure no cell is counted as its own neighbor.
      stopifnot(all(diag(adj) == 0))
      
      # --- 2. Get Neighborhood Compositions ---
      
      # Create a sparse matrix where rows are cells and columns are cell types.
      # This represents the "0-step" neighbors (the cell itself).
      counts <- Matrix::sparse.model.matrix(~ 0 + type_lvl3, .cells)
      
      # Expand the neighborhood by 'nsteps'.
      # Matrix multiplication aggregates neighbor counts at each step.
      for (iter in seq_len(nsteps)) {
        counts <- adj %*% counts
      }
      
      # Clean up column names for clarity.
      colnames(counts) <- gsub('type_lvl3', '', colnames(counts))
      
      # --- 3. Filter Cells and Neighbors for Modeling ---
      
      # Only consider specified neighbor types.
      valid_neighbor_types <- intersect(neighbor_types, colnames(counts))
      counts <- counts[, valid_neighbor_types, drop = FALSE]
      
      # Calculate the total number of relevant neighbors for each cell and log-transform it.
      # This will be used as an offset in the model to account for neighborhood density.
      .cells$log_n_neighbors <- log(rowSums(counts))
      
      # Filter out cells that have zero neighbors (where log(0) is -Inf).
      i <- which(!is.infinite(.cells$log_n_neighbors))
      M <- .cells[i, ]
      counts <- counts[i, , drop = FALSE]
      
      # Limit the analysis to the specified anchor cell types.
      i <- which(M$type_lvl3 %in% anchor_types)
      M <- M[i, ]
      counts <- counts[i, , drop = FALSE]
      
      # --- 4. Run Statistical Model (presto) ---
      
      # Suppress warnings that may arise during model fitting.
      suppressWarnings({
        # 'presto.presto' fits a fast generalized linear mixed model.
        # Formula: y ~ 1 + (1|type_lvl3) + offset(log_n_neighbors)
        # - y: The count of a specific neighbor type (handled internally by presto).
        # - (1|type_lvl3): A random intercept for each anchor cell type.
        # - offset(log_n_neighbors): Accounts for the total number of neighbors.
        presto_res <- presto::presto.presto(
          y ~ 1 + (1 | type_lvl3) + offset(log_n_neighbors),
          M,
          t(counts), # Counts matrix needs to be transposed for presto.
          'log_n_neighbors',
          effects_cov = c('type_lvl3'),
          nsim = 1000,
          ncore = 1, # Parallelism is handled by furrr at the sample level.
          family = 'poisson',
          min_sigma = .05,
          verbose = FALSE
        )
      })
      
      # --- 5. Calculate and Format Contrasts ---
      
      # Create a contrast matrix to compare anchor types.
      contrasts_mat <- make_contrast.presto(
        object = presto_res,
        var_contrast = 'type_lvl3'
      )
      contrasts_mat[is.na(contrasts_mat)] <- 0 # Replace NAs with 0.
      
      # Calculate the effects based on the contrast matrix.
      eff <- contrasts.presto(presto_res, contrasts_mat, one_tailed = FALSE) %>%
        dplyr::rename(anchor = contrast, neighbor = feature, logFC = beta, logSE = sigma) %>%
        dplyr::mutate(
          # Convert from natural log (ln) to log base 2 for easier interpretation.
          logFC = logFC / log(2),
          logSE = logSE / log(2)
        ) %>%
        data.table()
      
      return(eff)
      
    }, .options = furrr::furrr_options(seed = TRUE)) %>% # Use a seed for reproducibility.
    bind_rows(.id = 'SampleID') # Combine results from all samples into one data.table.
  
  return(res_all)
}


#' Summarize Enrichment Results with a Meta-Analysis
#'
#' This function takes the per-sample enrichment results and combines them using a
#' random-effects model with inverse variance weighting to produce a single,
#' summarized result for each anchor-neighbor pair.
#'
#' @param enrichment_results A data.table, typically the output from
#'   `calculate_spatial_enrichment`.
#' @param n_total The total number of samples, used in the REML prior calculation.
#' @param c_prior The prior value for the REML calculation.
#'
#' @return A final, summarized data.table with meta-analyzed logFC, p-values,
#'   and adjusted p-values for each anchor-neighbor interaction.
#'
summarize_enrichment_results <- function(enrichment_results, n_total = 8, c_prior = 0.1) {
  require(data.table)
  # This step groups the data by each anchor-neighbor pair and applies the
  # meta-analysis function to combine logFC and logSE values across all samples.
  res <- enrichment_results[, 
    combine_with_REML_prior(.SD$logFC, .SD$logSE, n_total = n_total, c_prior = c_prior),
    by = .(anchor, neighbor)
  ][
    order(pvalue) # Order results by p-value.
  ][
    !is.na(logFC) # Remove rows where logFC is NA.
  ][
    , padj := p.adjust(pvalue, 'BH') # Calculate Benjamini-Hochberg adjusted p-values.
  ]
  
  return(res)
}


# --- Meta-Analysis Helper Functions ---

#' Estimate Heterogeneity (Tau) via REML
#'
#' This function estimates the between-dataset standard deviation (tau), a measure of
#' heterogeneity, from observed effect sizes and their standard errors. It uses the
#' `rma.uni` function from the `metafor` package.
#'
#' @param est A numeric vector of effect size estimates (e.g., log-fold changes).
#' @param se A numeric vector of standard errors corresponding to the estimates.
#' @param method A character string specifying the method for estimating tau-squared.
#'   Defaults to "REML" (Restricted Maximum-Likelihood).
#' @return A numeric value representing the estimated between-study standard deviation (tau).
#'
tau_reml <- function(est, se, method = "REML") {
  # Ensure the estimates and standard errors are of the same length.
  stopifnot(length(est) == length(se))
  
  # Fit a random-effects meta-analysis model using the metafor package.
  # 'yi' are the effect sizes, and 'sei' are their standard errors.
  fit <- metafor::rma.uni(yi = est, sei = se, method = method)
  
  # Extract tau^2 (the variance) and take the square root to get tau (the standard deviation).
  as.numeric(sqrt(fit$tau2))
}

#' Combine Estimates with a Zero-Centered Prior for Missing Data
#'
#' This function performs a random-effects meta-analysis that incorporates a
#' zero-centered prior for "missing" studies. This is a form of regularization that
#' shrinks the overall estimate towards zero when data is sparse (i.e., when an
#' effect is observed in only a few out of many possible datasets).
#'
#' @param est A numeric vector of the *observed* effect size estimates.
#' @param se A numeric vector of the *observed* standard errors.
#' @param n_total An integer for the total number of datasets that *could* have
#'   contributed data (observed + missing).
#' @param tau_hat A numeric value for the estimated between-study heterogeneity (tau),
#'   typically from `tau_reml`.
#' @param c_prior A numeric scaling factor for the prior's standard deviation. The
#'   prior's SD is calculated as `tau0 = c_prior * tau_hat`. Defaults to 1.
#' @param se_floor An optional numeric value to prevent a too-strong (too narrow) prior
#'   when `tau_hat` is close to zero. The prior's SD will be `max(tau0, se_floor)`.
#' @return A data.table containing the meta-analyzed logFC, logSE, z-score, p-value,
#'   and estimates of tau.
#'
meta_with_zero_prior_RE <- function(est, se, n_total, tau_hat,
                                      c_prior = 1, se_floor = NULL) {
  # Perform sanity checks on the inputs.
  stopifnot(length(est) == length(se), n_total >= length(est))
  
  # Calculate the number of observed and missing studies.
  k_obs  <- length(est)
  k_miss <- n_total - k_obs
  
  # Calculate random-effects weights for the observed studies.
  # The weight for each study is the inverse of its variance (se^2 + tau_hat^2).
  w_obs <- 1 / (se^2 + tau_hat^2)
  w_obs_sum <- sum(w_obs)
  
  # Define the prior's standard deviation (tau0). The prior assumes missing studies
  # have an effect size of 0 with a standard deviation of tau0.
  tau0 <- c_prior * tau_hat
  
  # If an se_floor is provided, ensure the prior is never stronger (more certain)
  # than a typical observed SE. This prevents overly strong shrinkage if tau_hat is near zero.
  if (!is.null(se_floor)) {
    tau0 <- max(tau0, se_floor)
  }
  
  # Calculate the weight for each "missing" pseudo-study.
  w0 <- if (k_miss > 0) 1 / (tau0^2) else 0
  
  # Calculate the pooled estimate and its standard error.
  # This is a weighted average of the observed effects and the k_miss pseudo-studies at effect 0.
  w_tot <- w_obs_sum + k_miss * w0
  mu    <- if (w_tot > 0) sum(w_obs * est) / w_tot else NA_real_
  se_mu <- if (w_tot > 0) sqrt(1 / w_tot)       else NA_real_
  
  # Calculate z-score and two-tailed p-value.
  z     <- mu / se_mu
  p     <- 2 * (1 - pnorm(abs(z)))
  
  # Return the results in a data.table.
  data.table(
    logFC = mu, logSE = se_mu, zscore = z, pvalue = p,
    tau_hat = tau_hat, tau0 = tau0
  )
}

#' Wrapper to Estimate Tau and Perform Meta-Analysis with Prior
#'
#' This is a convenience wrapper that first estimates heterogeneity (tau) via REML and
#' then calls `meta_with_zero_prior_RE` to perform the meta-analysis. It includes
#' error handling for cases where tau cannot be estimated.
#'
#' @param est A numeric vector of observed effect size estimates.
#' @param se A numeric vector of observed standard errors.
#' @param n_total An integer for the total number of potential studies/samples.
#' @param c_prior A numeric scaling factor for the prior's standard deviation. Defaults to 1.
#' @param se_floor An optional minimum value for the prior's SD. If NULL (the default),
#'   it is set to the median of the observed standard errors.
#' @param method A character string for the method used by `tau_reml` to estimate tau-squared.
#' @return A data.table with meta-analysis results, or a data.table with NA values if
#'   an error occurs during calculation.
#'
combine_with_REML_prior <- function(est, se, n_total,
                                      c_prior = 1, se_floor = NULL,
                                      method = "REML") {
  
  # Handle cases with insufficient data for heterogeneity estimation.
  if (length(est) < 2) {
    # If there's only one observation, we cannot estimate between-study variance,
    # so we assume it's zero.
    tau_hat <- 0
    if (is.null(se_floor)) se_floor <- stats::median(se, na.rm = TRUE)
    
    return(meta_with_zero_prior_RE(est, se, n_total, tau_hat, c_prior = c_prior, se_floor = se_floor))
  }
  
  # Use tryCatch for the standard case with >= 2 observations, as convergence can still fail.
  tryCatch({
    
    # Estimate between-study heterogeneity (tau) from the observed data.
    tau_hat <- tau_reml(est, se, method = method)
    
    # Set a default for se_floor if not provided. Using the median observed SE
    # is a reasonable heuristic to prevent the prior from being overly strong.
    if (is.null(se_floor)) se_floor <- stats::median(se, na.rm = TRUE)
    
    # Perform the meta-analysis using the estimated tau.
    meta_with_zero_prior_RE(est, se, n_total, tau_hat, c_prior = c_prior, se_floor = se_floor)
    
  }, error = function(e) {
    # If any error occurs, return a data.table with NA values.
    # This prevents the entire analysis pipeline from crashing.
    message("REML failed to converge for a group. Details: ", e$message)
    data.table(logFC=NA_real_, logSE=NA_real_, zscore=NA_real_, pvalue=NA_real_, tau_hat=NA_real_, tau0=NA_real_)
  })
}

#' @title Calculate Cell Counts in Distance Bins from an Interface
#' @description This function takes spatial coordinates of cells and interface lines,
#'   calculates the signed distance of each cell to the nearest interface, and
#'   groups cells into discrete distance bins. It returns a matrix of cell
#'   type counts per bin for a single sample.
#' @param cells A data.table containing cell information, including 'X'/'Y' coordinates,
#'   cell type ('type_lvl3'), and a region annotation ('tessera_annotation').
#' @param interfaces An sf object containing interface geometries (e.g., LINESTRINGs).
#' @return A matrix where rows are distance bins (e.g., "(-5,0]") and columns
#'   are cell types ('type_lvl3'), with values representing cell counts.
get_bins_per_cell = function(cells, interfaces) {
    ## Get distances and closest interface type
    pts = st_as_sf(cells[, .(X, Y)], coords = c('X', 'Y'))
    geos_pts = geos::as_geos_geometry(pts$geometry)
    geos_lines = geos::as_geos_geometry(interfaces$x[1:nrow(interfaces)])
    
    nearest_interfaces_idx = geos::geos_nearest(geos_pts, geos_lines)
    
    cells$closest_interface_type = interfaces$Type_of_Interface[nearest_interfaces_idx]
    cells$dist_interface = geos::geos_distance(geos_pts, geos_lines[nearest_interfaces_idx])
    
    ## Assign sign to distances
    cells$dist_interface_signed = fifelse(
        cells$tessera_annotation == 'Stromal-enriched',
        -cells$dist_interface,
        cells$dist_interface
    )
    
    ## Assign cells to 5um bins
    dist_breaks = seq(-100, 100, by = 5)
    cells$dist_bin = cut(cells$dist_interface_signed, breaks = dist_breaks, include.lowest = TRUE)

    # --- ROBUST SUMMARIZATION --
    # NOTE: THIS IS DIFFERENT FROM THIS FUNCTION IN OTHER NOTEBOOKS
    # cells = cells %>% filter(
    #     (closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' & cxcl_pos_tile == 'CXCL_pos') | (closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' & cxcl_pos_tile == 'CXCL_neg')        
    # )
    
    cells_in_range = cells[!is.na(dist_bin)]
    
    if (nrow(cells_in_range) == 0) {
        warning("No cells found within the -100 to 100µm distance range.")
        return(list())
    }

    all_interface_types = unique(cells$closest_interface_type)
    cells_in_range[, closest_interface_type := factor(closest_interface_type, levels = all_interface_types)]

    # counts_long = cells_in_range[, .N, by = .(closest_interface_type, dist_bin, type_lvl3)]

    # counts_wide = dcast(counts_long,
    #                     closest_interface_type + dist_bin ~ type_lvl3,
    #                     value.var = "N",
    #                     fill = 0,
    #                     drop = FALSE)

    # result_list = split(counts_wide, by = "closest_interface_type")

    # result_list = lapply(result_list, function(dt) {
    #     row_names = dt$dist_bin
    #     count_cols = setdiff(names(dt), c("closest_interface_type", "dist_bin"))
    #     mat = as.matrix(dt[, ..count_cols])
    #     rownames(mat) = row_names
    #     return(mat)
    # })

    return(cells)
}

In [ ]:
# Load interface geometry files for each sample
ids = unique(tile_metadata$SampleID[tile_metadata$Status == 'MSI'])
interfaces = map(ids, function(.id) {
    fname = normalizePath(list.files(path = '../Tessera tiles/Spatial objects for tumor-stromal interfaces in all MERFISH samples/', pattern = '_tumor_stromal_interfaces.rds', full.names = TRUE)[grepl(list.files(path = '../Tessera tiles/Spatial objects for tumor-stromal interfaces in all MERFISH samples/', pattern = '_tumor_stromal_interfaces.rds', full.names = TRUE), pattern = .id)])
    readRDS(fname)
})
names(interfaces) = ids

glimpse(interfaces[[1]])

In [ ]:
# Set a higher limit for global variables when using parallel processing
plan(sequential)
plan(multicore)
options(future.globals.maxSize = 1e10)

# Run get_bins for each sample in parallel
system.time({
    distances = future_map(ids, function(.id) {
        get_bins_per_cell(tile_metadata[SampleID == .id], interfaces[[.id]])    
    }, .options = furrr::furrr_options(seed=TRUE))
    names(distances) = ids
})
distances = rbindlist(distances) %>% na.omit()
head(distances) 
plan(sequential)

In [ ]:
tiles_to_omit = read.csv('../Tessera tiles/Tessera processed results/tiles_to_exclude_from_interface_analysis.csv') %>%
    filter(tiles_to_exclude_from_interface_analysis != '') %>%
    pull(agg_id)
length(tiles_to_omit)
head(tiles_to_omit)

In [ ]:
distances$dist_interface_signed %>% head

In [ ]:
# --- Data Import and Processing ---
distances = distances %>%
    filter(! agg_id %in% tiles_to_omit) %>%
    mutate(measurement_bin = factor(dist_bin)) %>%
    mutate(measurement_bin = fct_reorder(measurement_bin, dist_interface_signed, .fun = mean)) %>%
    filter(MMRstatus == 'MMRd') %>%
    mutate(dist = dist_interface_signed)
distances$type_lvl1[distances$type_lvl2 == 'Mast'] = 'Mast'

# --- Inspect the Result ---

glimpse(distances)

In [ ]:
distances_trimmed = distances %>% filter(abs(dist) <= 100) # remove distant cells
dim(distances_trimmed)
glimpse(distances_trimmed)

# read in harmonized data

In [ ]:
merged_merfish = readr::read_rds('../Harmony and UMAP embeddings of MERFISH cells/harmonized_merfish_20241105.rds')
merged_merfish

In [ ]:
merged_merfish@meta.data$orig.ident[merged_merfish@meta.data$orig.ident %in% c('G4659-CP-MET', 'G4659-CP-MET_VMSC04701')] = 'G4659'

In [ ]:
unique(merged_merfish@meta.data$orig.ident)

In [ ]:
head(merged_merfish@meta.data)

In [ ]:
.counts = GetAssayData(merged_merfish, layer = 'counts') 
dim(.counts)

In [ ]:
.total_counts = data.frame(total_RNA = colSums(.counts)) %>%
    tibble::rownames_to_column('cell_id')

In [ ]:
dim(.total_counts)

# Show donor-level traces and a median line for multiple genes at the same time: hub+ MSI vs hub- MSI interfaces

In [ ]:
# Load the data.table library for efficient data manipulation
library(data.table)

# Define the list of genes and associated comments
# Create a data.table directly, which is more efficient than creating a data.frame
# Use the := operator to add the 'location' column by reference
gene_dt <- data.table(
  gene_name = c(
    "CXCL9", "CXCL10", "CXCL11", "CXCL13", "CCL5", "CCL18", "CXCL16",
    "CCL19", "CCL21", "ZNF683", "ITGAE", "PDCD1", "TGFB1", "CD274",
    "IDO1", "CCL3", "CCL4", "CCL17", "CCL22", "IFNG", "PDCD1LG2",
    "TGFB2", "TGFB3", "IL10", "ENTPD1", "NT5E", "CD38", "PTGS2",
    "PTGES2", "CCR4", "CCR5", "CCR7", "CCR8", "CXCR3", "CXCR5",
    "CXCR6", "LAG3", "PTGES", "PTGS1"
  ),
  comment = c(
    "ligand - good", "ligand - good", "ligand - good", "ligand - good",
    "(CD8 T cells) - ligand - good", "(myeloid ISG) - ligand - good",
    "ligand - good", "ligand - good", "ligand - good",
    "transcription factor - good", "receptor for CDH1 - good.", NA,
    "ligand - good", "(PDL1)- ligand - good", "ligand - good",
    "ligand - good", "ligand - good", "(mregDC) - ligand - bad",
    "(mregDC) - ligand - bad", "ligand - bad", NA, "ligand - medium",
    "ligand - good - supp", NA, "(CD39) ligand - bad",
    "(CD73) - ligand - medium", NA, "ligand - medium", "ligand - medium",
    "(Tregs) - receptor - bad", "receptor - bad", "receptor - bad",
    "(Tregs) - receptor - bad", "receptor - bad", "receptor - medium",
    "receptor - bad", "receptor", "enzyme", "enzyme"
  )
)

# Add the 'location' column. Default to 'supplement', set the first 15 to 'main',
# then specifically set PDCD1 (gene #12) back to 'supplement'.
gene_dt[, location := "supplement"]
gene_dt[1:15, location := "main"]
gene_dt[gene_name == "PDCD1", location := "supplement"]


# --- Data Preparation ---
# Extract all gene names for subsequent filtering and reshaping
all_gene_names <- gene_dt[, gene_name]

# Check how many of these genes are present in the counts matrix
# This is for verification and does not alter data
# sum(all_gene_names %in% rownames(.counts))

# Subset the counts matrix for the genes of interest, transpose it,
# and convert to a data.table.
# `keep.rownames = "cell"` efficiently converts row names to a 'cell' column.
temp_counts_dt <- as.data.table(
  t(.counts[all_gene_names, ]),
  keep.rownames = "cell"
)

# Calculate total transcript counts per cell and store in a data.table
total_counts_dt <- data.table(
  cell = names(colSums(.counts)),
  total_counts = colSums(.counts)
)

## --- Merging Data Tables ---
# Convert the Seurat object metadata to a data.table.
meta_dt <- as.data.table(merged_merfish@meta.data)

# Rename 'orig.ident' to 'PatientID' by reference using setnames for efficiency.
setnames(meta_dt, "orig.ident", "PatientID")

# Convert the distances data.frame to a data.table.
distances_dt <- as.data.table(distances_trimmed)

meta_dt = meta_dt %>% filter(PatientID %in% unique(tile_metadata$PatientID[tile_metadata$Status == 'MSI']))

# Select and rename columns in the distances data.table.
# Note: Updated to use 'tessera_annotation' as per your error message.
distances_dt <- distances_dt[, .(
  cell = cell_id, dist, measurement_bin,
  SampleID, PatientID, tessera_annotation, closest_interface_type
)]
distances_dt[, `:=`(
  # Parse the measurement_bin string '(lower,upper]' to extract numeric bounds.
  upper_bound = as.numeric(gsub(measurement_bin, pattern = ".*,|)|]", replacement = "")),
  lower_bound = as.numeric(gsub(measurement_bin, pattern = "\\[|\\(|\\)|,.*", replacement = ""))
)]

distances_dt[, `:=`(
  # Assign the bin midpoint.
  bin_numeric = (upper_bound + lower_bound)/2
)]

# Sequentially left-join the data.tables.
temp_dt <- merge(meta_dt, temp_counts_dt, by = "cell", all.x = TRUE)
temp_dt <- merge(temp_dt, total_counts_dt, by = "cell", all.x = TRUE)

# The corrected merge call, joining ONLY on the shared columns.
temp_dt <- merge(temp_dt, distances_dt, by = c("cell", "PatientID"), all.x = TRUE)

# Filter out rows where the distance is missing
temp_dt <- temp_dt[!is.na(dist)]

# Clean up intermediate objects to free up memory
rm(temp_counts_dt, merged_merfish)
gc()


# --- Data Reshaping and Final Calculations ---
# Reshape the data from wide to long format using melt().
# This is the data.table equivalent of tidyr::pivot_longer.
# `id.vars` are the columns to keep, `measure.vars` are the columns to pivot.
temp2_dt <- melt(
  temp_dt,
  id.vars = setdiff(names(temp_dt), all_gene_names),
  measure.vars = all_gene_names,
  variable.name = "gene",
  value.name = "counts"
)

# Add new columns by reference using the `:=` operator.
# This single step performs all calculations efficiently without creating a copy.
temp2_dt[, `:=`(
  # Calculate the proportion of transcripts for each gene in each cell.
  propTx = counts / total_counts,
  # Parse the measurement_bin string '(lower,upper]' to extract numeric bounds.
  upper_bound = as.numeric(gsub(measurement_bin, pattern = ".*,|)|]", replacement = "")),
  lower_bound = as.numeric(gsub(measurement_bin, pattern = "\\[|\\(|\\)|,.*", replacement = "")),
  # Assign the bin midpoint.
  midpoint = (upper_bound + lower_bound)/2
)]
# %>%
#     mutate(Bins = cut(x = dist, breaks = seq(min(dist), max(dist), 3), include.lowest = TRUE, ordered_result = TRUE)) %>%
#         mutate(upper_bound = as.numeric(gsub(Bins, pattern = '.*,|)|]', replacement = "")), 
#         lower_bound = gsub(.data$Bins, pattern = '\\[|\\(|\\)|,.*', replacement = "") %>% as.numeric,
#         midpoint = (upper_bound + lower_bound)/2
#         ) %>%
# View the first few rows of the final data.table
head(temp2_dt)

In [ ]:
temp2_dt$SampleID %>% unique

# This code iterates through a list of gene names and, for each gene, calculates summary statistics on its proportional expression across different experimental conditions (interface, spatial bins, etc.).


In [ ]:
unique(temp2_dt$interface_type)

In [ ]:
# The final output is a list of data frames, one for each gene.
summary_stats_list <- lapply(all_gene_names, function(.gene, .data, .temp2) {

  # Print the name of the gene currently being processed for progress tracking.
  message("Processing gene: ", .gene)

  # --- 1. Data Preparation and Merging ---

  # Start with the '.temp2' dataframe, which contains gene count information.
  # Filter it to keep only the data for the current gene.
  # Then, join it with patient status information from the '.data' dataframe.
  .temp3 <- .temp2 %>%
    filter(gene == .gene) 

  # --- 2. Calculate Proportional Expression per Patient ---

  # Aggregate counts for each patient within each experimental bin.
  # This step calculates the total counts for the specific gene and the overall
  # total counts within that bin for each patient.
  .temp4 <- .temp3 %>%
    mutate(measurement_bin = as.factor(measurement_bin)) %>%
    group_by(PatientID,closest_interface_type, measurement_bin, midpoint, gene) %>%
    summarize(
      total_gene_counts = sum(counts, na.rm = TRUE),
      total_counts = sum(total_counts, na.rm = TRUE),
      .groups = 'drop'
    ) %>%
    # Calculate the proportion of the specific gene's transcript (Tx)
    # relative to all transcripts in that bin.
    mutate(prop_Tx = total_gene_counts / total_counts)

  # --- 3. Calculate Summary Statistics Across Patients ---

  # Now, calculate the mean, standard deviation, and standard error of the
  # proportional expression across all patients for each condition.
  summary_df <- .temp4 %>%
    group_by(closest_interface_type, gene, measurement_bin, midpoint) %>%
    summarise(
      # Calculate the average proportion across patients.
      mean_prop = mean(prop_Tx, na.rm = TRUE),
      # Calculate the standard deviation.
      sd_prop = sd(prop_Tx, na.rm = TRUE),
      # Count the number of patients in the group.
      n_patients = n(),
      # Calculate the standard error of the mean (SEM).
      sem_prop = sd_prop / sqrt(n_patients),
      .groups = 'drop'
    ) %>%
    # Calculate the upper and lower bounds for error bars.
    mutate(
      ymin = mean_prop - sem_prop,
      ymax = mean_prop + sem_prop
    ) %>%
    as.data.frame() # Convert to a standard data frame.

  # --- 4. Return the Final Data Frame ---

  return(summary_df)

}, .data = distances_trimmed, .temp2 = temp2_dt) # Pass the necessary data frames to the function
summary_stats = rbindlist(summary_stats_list)
sample_n(summary_stats, 10)
summary_stats %>% fwrite('gene_expression_spatial_patterns_hubPos_vs_hubNeg.csv')

In [ ]:
summary_stats %>% sample_n(20)

# Test for spatial gene patterning with wilcoxon

In [ ]:
head(temp2_dt)

In [ ]:
unique(temp2_dt$SampleID)

In [ ]:

# The final output is a list of data frames, one for each gene.
summary_stats_list <- lapply(all_gene_names, function(.gene, .temp2) {

  # Print the name of the gene currently being processed for progress tracking.
  message("Processing gene: ", .gene)

  # --- 1. Data Preparation and Merging ---

  # Start with the '.temp2' dataframe, which contains gene count information.
  # Filter it to keep only the data for the current gene.
  # Then, join it with patient status information from the '.data' dataframe.
  .temp3 <- .temp2 %>%
    filter(gene == .gene) #%>%
    # # Create the 'interface' column based on the 'Status'.
    # mutate(interface = case_when(
    #   Status == 'MSI' ~closest_interface_type,
    #   Status == 'MSS' ~ 'MSS',
    #   TRUE ~ NA_character_ # Good practice to handle unexpected cases
    # ))

  # --- 2. Calculate Proportional Expression per Patient ---

  # Aggregate counts for each patient within each experimental bin.
  # This step calculates the total counts for the specific gene and the overall
  # total counts within that bin for each patient.
  .temp4 <- .temp3 %>%
    mutate(measurement_bin = as.factor(measurement_bin)) %>%
    group_by(PatientID,closest_interface_type, measurement_bin, midpoint, gene) %>%
    summarize(
      total_gene_counts = sum(counts, na.rm = TRUE),
      total_counts = sum(total_counts, na.rm = TRUE),
      .groups = 'drop'
    ) %>%
    # Calculate the proportion of the specific gene's transcript (Tx)
    # relative to all transcripts in that bin.
    mutate(prop_Tx = total_gene_counts / total_counts)


  return(.temp4)

},  .temp2 = temp2_dt) # Pass the necessary data frames to the function

In [ ]:
complete_gene_data = rbindlist(summary_stats_list)
fwrite(complete_gene_data, 'complete_gene_data_hubPos_vs_hubNeg.csv')
head(complete_gene_data)

In [ ]:
unique(complete_gene_data$interface_type)

In [ ]:
# Filter the data to include only the 'Hub+' and 'Hub-'closest_interface_type groups
# Also, ensure 'interface_type' is a factor with the desired order for plotting
paired_data <- complete_gene_data %>%
  filter(closest_interface_type %in% c("Hub+", "Hub-"))

paired_data_Hub = paired_data %>%
  filter(closest_interface_type == 'Hub+') %>%
  group_by(measurement_bin, gene, midpoint) %>%
  complete(PatientID,closest_interface_type, fill = list(total_gene_counts = 0, total_counts = 0, prop_Tx = 0)) %>%
  ungroup()

paired_data_nonHub = paired_data %>%
  filter(closest_interface_type == 'Hub-') %>%
  group_by(measurement_bin, gene, midpoint) %>%
  complete(PatientID,closest_interface_type, fill = list(total_gene_counts = 0, total_counts = 0, prop_Tx = 0)) %>%
  ungroup()

paired_data = rbind(paired_data_Hub, paired_data_nonHub) %>%
  mutate(closest_interface_type = factor(closest_interface_type, levels = c("Hub-", "Hub+"))) 
# Display the first few rows of the prepared data
print("Prepared Data Head:")
head(paired_data)

In [ ]:
paired_data$PatientID %>% unique %>% length

## Perform Paired Wilcoxon Test, Calculate Log2FC, and Apply fdr Correction

In [ ]:

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Step 3: Perform Paired Wilcoxon Test, Calculate Log2FC, and Apply BH Correction
#
# We will group by Gene and Bin, run the test, calculate the log2 fold change,
# and then apply the Benjamini-Hochberg correction across all tests.
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Group by Gene and Bins, then perform the Wilcoxon test and calculate Log2FC
stat_test_results_raw <- paired_data %>%
  group_by(gene, measurement_bin) %>%
  do({
    # Capture the current data subset
    current_data <- .
    
    # Initialize p-value and log2fc
    p_value <- NA
    log2fc <- NA
    
    # Use a try-catch block in case the test fails for a group
    test_result <- try(wilcox.test(prop_Tx ~closest_interface_type, data = current_data, paired = TRUE), silent = FALSE)
    
    if (!inherits(test_result, "try-error")) {
      p_value <- test_result$p.value
    }
    
    # --- NEW: Calculate Log2 Fold Change ---
    # Calculate mean proportions for each group, adding a small pseudocount
    mean_cxcl_pos <- mean(current_data$prop_Tx[current_data$interface_type == "Hub+"], na.rm = TRUE) + 1e-6
    mean_cxcl_neg <- mean(current_data$prop_Tx[current_data$interface_type == "Hub-"], na.rm = TRUE) + 1e-6
    
    # Calculate log2 fold change
    log2fc <- log2(mean_cxcl_pos / mean_cxcl_neg)
    
    # Return a tidy data frame with p-value and log2fc
    data.frame(p = p_value, log2fc = log2fc)
  }) %>%
  ungroup() %>%
  # Remove rows where the test failed
  filter(!is.na(p))

# Apply Benjamini-Hochberg correction to the collected p-values
stat_test_results_wilcox <- stat_test_results_raw %>%
  mutate(p.adj = p.adjust(p, method = "fdr")) %>%
  # Add other necessary columns for plotting
  mutate(group1 = "Hub-", group2 = "Hub+") 

# Display the statistical test results
print("Statistical Test Results with FDR-Adjusted P-values and Log2FC:")
print(stat_test_results_wilcox)


## Display the statistical test results

In [ ]:
print("Statistical Test Results with FDR-Adjusted P-values and Log2FC:")
head(stat_test_results_wilcox)

## Plot all genes

### Prepare data for the shaded significance rectangles (geom_rect)

In [ ]:
stat_test_results_wilcox

In [ ]:
pvalues_for_rects <- stat_test_results_wilcox %>%
  mutate(fdr = p.adjust(p, 'fdr')) %>%
  left_join(., complete_gene_data %>% select(measurement_bin, midpoint) %>% distinct) %>%
  mutate(midpoint = as.numeric(as.vector(midpoint))) %>%
  mutate(is_significant = fdr < 0.1 & abs(log2fc) > 1) %>% #  & log2_fold_change < -0.585
  filter(is_significant == TRUE) 

if (nrow(pvalues_for_rects) > 0) {
    pvalues_for_rects = pvalues_for_rects %>%
      group_by(gene) %>%
      mutate(block = cumsum(is_significant != lag(is_significant, default = first(is_significant)))) %>%
      group_by(gene, block, is_significant) %>%
      summarise(
        min_midpoint = min(midpoint),
        max_midpoint = max(midpoint),
        .groups = 'drop'
      )
}
pvalues_for_rects #%>% sample_n(10)

## Plot all genes

In [ ]:
head(summary_stats)

In [ ]:
summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic') %>% head

In [ ]:
fig.size(height = 9, width = 16, res = 400)
gene_plot = ggplot(data = summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic'), aes(x = midpoint,
                    y = mean_prop,
                    color = closest_interface_type)) +
    geom_line(alpha = 1) +
    geom_ribbon(
            aes(
                ymin = ymin,
                ymax = ymax,
                fill =closest_interface_type), alpha = 0.25, inherit.aes = TRUE) +
    scale_y_continuous(labels = scales::percent) +
    scale_color_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", "Heterotypic" = 'grey')
  ) +
  scale_fill_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", 'Heterotypic' = 'grey')
  ) +
 # Add labels and facet by celltype
  labs(
    x = "Microns from interface",
    y = "Percent of all transcripts"
  ) +
  geom_vline(xintercept = 0, color = 'red', linetype = 'dashed') + 
  # Final theme adjustments
  cowplot::theme_half_open(10) + 
  theme(
    legend.position = "top",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 7)
  ) +
  guides(color = guide_legend(override.aes = list(shape = 16, size = 4, alpha = 1))) +
  facet_wrap(~gene, scales = 'free')
gene_plot

pdf('spatial_gene_patterning.pdf', width = 16, height = 9)
gene_plot
dev.off()

In [ ]:
pvalues_for_rects <- stat_test_results_wilcox %>%
  mutate(fdr = p.adjust(p, 'fdr')) %>%
  left_join(., complete_gene_data %>% select(measurement_bin, midpoint) %>% distinct) %>%
  mutate(midpoint = as.numeric(as.vector(midpoint))) %>%
  mutate(is_significant = fdr < 0.1 & abs(log2fc) > 1) %>% #  & log2_fold_change < -0.585
  filter(is_significant == TRUE) 

if (nrow(pvalues_for_rects) > 0) {
    pvalues_for_rects = pvalues_for_rects %>%
      group_by(gene) %>%
      mutate(block = cumsum(is_significant != lag(is_significant, default = first(is_significant)))) %>%
      group_by(gene, block, is_significant) %>%
      summarise(
        min_midpoint = min(midpoint),
        max_midpoint = max(midpoint),
        .groups = 'drop'
      )
}
pvalues_for_rects #%>% sample_n(10)

In [ ]:
summary_stats = fread('gene_expression_spatial_patterns_hubPos_vs_hubNeg.csv')
head(summary_stats)

In [ ]:
fig.size(height = 9, width = 16, res = 400)

gene_plot = ggplot(data = summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic'), aes(x = midpoint,
                    y = mean_prop,
                    color =closest_interface_type)) +
    geom_line(alpha = 1, linewidth = 0.25) +
    geom_ribbon(
            aes(
                ymin = ymin,
                ymax = ymax,
                fill =closest_interface_type), color = NA, alpha = 0.25, inherit.aes = TRUE) +
    scale_y_continuous(labels = scales::percent) +
    scale_color_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", "MSS" = 'grey')
  ) +
  scale_fill_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", 'MSS' = 'grey')
  ) +
 # Add labels and facet by celltype
  labs(
    x = expression(paste("Distance from interface (", mu, "m)")),
    y = "Percent of all transcripts"
  ) +
  geom_vline(xintercept = 0, color = 'red', linetype = 'dotted') + 
  # Final theme adjustments
  cowplot::theme_half_open(7) + 
  theme(
    legend.position = "top",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 7)
  ) +
  guides(color = guide_legend(override.aes = list(shape = 16, size = 4, alpha = 1))) +
  facet_wrap(~gene, scales = 'free') #+
  # Add shaded rectangles for significant regions
  # geom_rect(
  #   data = pvalues_for_rects,
  #   aes(xmin = min_midpoint, xmax = max_midpoint, ymin = -Inf, ymax = Inf),
  #   fill = "#EEE8AA",
  #   color = "#EEE8AA",
  #   alpha = 0.5,
  #   inherit.aes = FALSE
  # ) 
gene_plot + ggtitle('Paired Wilcoxon test') + labs(subtitle = 'Yellow: fdr < 0.1 & abs(log2fc) > 1')

# Main figure

In [ ]:
fig.size(height = 5, width = 6, res = 400)
main_fig_genes = gene_vector <- c(
  "CXCL9",
  "CXCL10",
  "CXCL11",
  "CXCL13",
  "CCL5",
  "CCL18",
  "CXCL16",
  "CCL19",
  "CCL21",
  "ZNF683",
  "ITGAE",
  "TGFB1",
  "IDO1",
  "CD274", 
  "PTGS2",
  "PTGES2"
) #gene_dt %>% filter(location == 'main') %>% pull(gene_name)
main_fig_genes
gene_plot = ggplot(data = summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic')%>%
    filter(gene %in% main_fig_genes) %>%
    mutate(gene = factor(gene, levels = main_fig_genes), ordered = TRUE), aes(x = midpoint,
                    y = mean_prop,
                    color =closest_interface_type)) +
    geom_line(alpha = 1, linewidth = 0.25) +
    geom_ribbon(
            aes(
                ymin = ymin,
                ymax = ymax,
                fill =closest_interface_type), color = NA, alpha = 0.25, inherit.aes = TRUE) +
    scale_y_continuous(labels = scales::percent) +
    scale_color_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", "MSS" = 'grey')
  ) +
  scale_fill_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", 'MSS' = 'grey')
  ) +
 # Add labels and facet by celltype
  labs(
    x = expression(paste("Distance from interface (", mu, "m)")),
    y = "Percent of all transcripts"
  ) +
  geom_vline(xintercept = 0, color = 'red', linetype = 'dotted') + 
  # Final theme adjustments
  cowplot::theme_half_open(7) + 
  theme(strip.background = element_rect(fill = NA),
    legend.position = "top",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 7)
  ) +
  guides(color = guide_legend(override.aes = list(shape = 16, size = 4, alpha = 1))) +
  facet_wrap(~gene, scales = 'free') +
  # # Add shaded rectangles for significant regions
  # geom_rect(
  #   data = pvalues_for_rects %>% filter(gene %in% main_fig_genes)%>%
  #   mutate(gene = factor(gene, levels = main_fig_genes), ordered = TRUE),
  #   aes(xmin = min_midpoint, xmax = max_midpoint, ymin = -Inf, ymax = Inf),
  #   fill = "#EEE8AA",
  #   color = "#EEE8AA",
  #   alpha = 0.5,
  #   inherit.aes = FALSE
  # ) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    color = guide_legend(override.aes = list(shape = 16, size = 4)),
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  new_scale_fill() +
  # --- NEW: Add a dummy geom_point for the significance legend ---
  # This point won't be plotted (NA coordinates) but its aesthetic will appear in the legend.
  geom_point(
    data = data.frame(fill = "FDR < 0.1 & absolute log2 FC > 1"), # The text for the legend
    aes(fill = fill, x = 0, y = 0),
    shape = '.',
    size = 0,
    inherit.aes = FALSE
  ) +
  scale_fill_manual(
    name = "", # Set the same name for color and fill to merge legends
    values = c(
        "FDR < 0.1 & absolute log2 FC > 1" = "#EEE8AA" # Assign yellow to the new item
    )) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  geom_vline(xintercept = 0, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = -50, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = 50, color = 'grey', linetype = 'dotted') +
  NULL
gene_plot

pdf('spatial_gene_patterning_main_figure.pdf', width = 6, height = 5)
gene_plot
dev.off()

# Supplement - ligands

In [ ]:
gene_dt %>% filter(location == 'supplement')

In [ ]:
gene_dt$comment[gene_dt$gene_name == 'CD38'] = 'receptor'
gene_dt$comment[gene_dt$gene_name == 'IL10'] = 'ligand'
gene_dt$comment[gene_dt$gene_name == 'PDCD1LG2'] = 'ligand'
gene_dt$comment[gene_dt$gene_name == 'PDCD1'] = 'receptor'
gene_dt$location[gene_dt$gene_name == 'PTGS2'] = 'main'
gene_dt$location[gene_dt$gene_name == 'PTGES2'] = 'main'

In [ ]:
fig.size(height = 4, width = 6, res = 400)
supplementary_ligands = c(
  "CCL3",
  "CCL4",
  "CCL17",
  "CCL22",
  "IFNG",
  "PDCD1LG2",
  "TGFB2",
  "TGFB3",
  "PTGES",
  "PTGS1",
  "IL10",
  "ENTPD1",
  "NT5E"
)#gene_dt %>% filter(location == 'supplement' & grepl(comment, pattern = 'ligand|enzyme')) %>% pull(gene_name)
supplementary_ligands
gene_plot_supplementary_ligands = ggplot(data = summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic')%>%
    filter(gene %in% supplementary_ligands) %>%
    mutate(gene = factor(gene, levels = supplementary_ligands)), aes(x = midpoint,
                    y = mean_prop,
                    color =closest_interface_type)) +
    geom_line(alpha = 1, linewidth = 0.25) +
    geom_ribbon(
            aes(
                ymin = ymin,
                ymax = ymax,
                fill =closest_interface_type), color = NA, alpha = 0.25, inherit.aes = TRUE) +
    scale_y_continuous(labels = scales::percent) +
    scale_color_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", "MSS" = 'grey')
  ) +
  scale_fill_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", 'MSS' = 'grey')
  ) +
 # Add labels and facet by celltype
  labs(
    x = expression(paste("Distance from interface (", mu, "m)")),
    y = "Percent of all transcripts"
  ) +
  geom_vline(xintercept = 0, color = 'red', linetype = 'dotted') + 
  # Final theme adjustments
  cowplot::theme_half_open(7) + 
  theme(
    legend.position = "top",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 7)
  ) +
  guides(color = guide_legend(override.aes = list(shape = 16, size = 4, alpha = 1))) +
  facet_wrap(~gene, scales = 'free')+
  # Add shaded rectangles for significant regions
  # geom_rect(
  #   data = pvalues_for_rects %>% filter(gene %in% supplementary_ligands)%>%
  #   mutate(gene = factor(gene, levels = supplementary_ligands)),
  #   aes(xmin = min_midpoint, xmax = max_midpoint, ymin = -Inf, ymax = Inf),
  #   fill = "#EEE8AA",
  #   color = "#EEE8AA",
  #   alpha = 0.5,
  #   inherit.aes = FALSE
  # ) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    color = guide_legend(override.aes = list(shape = 16, size = 4)),
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  new_scale_fill() +
  # --- NEW: Add a dummy geom_point for the significance legend ---
  # This point won't be plotted (NA coordinates) but its aesthetic will appear in the legend.
  geom_point(
    data = data.frame(fill = "FDR < 0.1 & absolute log2 FC > 1"), # The text for the legend
    aes(fill = fill, x = 0, y = 0),
    shape = '.',
    size = 0,
    inherit.aes = FALSE
  ) +
  scale_fill_manual(
    name = "", # Set the same name for color and fill to merge legends
    values = c(
        "FDR < 0.1 & absolute log2 FC > 1" = "#EEE8AA" # Assign yellow to the new item
    )) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  geom_vline(xintercept = 0, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = -50, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = 50, color = 'grey', linetype = 'dotted') +
  NULL
gene_plot_supplementary_ligands

pdf('spatial_gene_patterning_supplement_ligands.pdf', width = 6, height = 4)
gene_plot_supplementary_ligands
dev.off()

## Supplement - receptors

In [ ]:
gene_dt %>%
    filter(location == 'supplement')

In [ ]:
fig.size(height = 4, width = 6, res = 400)
supplementary_receptors = c(
  "CD38",
  "PDCD1",
  "CCR4",
  "CCR5",
  "CCR7",
  "CCR8",
  "CXCR3",
  "CXCR5",
  "CXCR6",
  "LAG3"
) #gene_dt %>% filter(location == 'supplement' & grepl(comment, pattern = 'receptor')) %>% pull(gene_name)
supplementary_receptors
gene_plot_supplementary_receptors = ggplot(data = summary_stats %>% 
                   mutate(closest_interface_type = case_when(
                        closest_interface_type == 'CXCLneg tumor & CXCLneg stroma' ~ 'Hub-',
                        closest_interface_type == 'CXCLpos tumor & CXCLpos stroma' ~ 'Hub+',
                       .default = 'Heterotypic'
                   )) %>%
                   filter(!closest_interface_type == 'Heterotypic')%>%
    filter(gene %in% supplementary_receptors) %>%
    mutate(gene = factor(gene, levels = supplementary_receptors)), aes(x = midpoint,
                    y = mean_prop,
                    color =closest_interface_type)) +
    geom_line(alpha = 1, linewidth = 0.25) +
    geom_ribbon(
            aes(
                ymin = ymin,
                ymax = ymax,
                fill =closest_interface_type), color = NA, alpha = 0.25, inherit.aes = TRUE) +
    scale_y_continuous(labels = scales::percent) +
    scale_color_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", "MSS" = 'grey')
  ) +
  scale_fill_manual(
    name = "Interfaces:",
    values = c("Hub+" = "#D55E00", "Hub-" = "#009E73", 'MSS' = 'grey')
  ) +
 # Add labels and facet by celltype
  labs(
    x = expression(paste("Distance from interface (", mu, "m)")),
    y = "Percent of all transcripts"
  ) +
  geom_vline(xintercept = 0, color = 'red', linetype = 'dotted') + 
  # Final theme adjustments
  cowplot::theme_half_open(7) + 
  theme(
    legend.position = "top",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 7)
  ) +
  guides(color = guide_legend(override.aes = list(shape = 16, size = 4, alpha = 1))) +
  facet_wrap(~gene, scales = 'free')+
  # Add shaded rectangles for significant regions
  # geom_rect(
  #   data = pvalues_for_rects %>% filter(gene %in% supplementary_receptors)%>%
  #   mutate(gene = factor(gene, levels = supplementary_receptors)),
  #   aes(xmin = min_midpoint, xmax = max_midpoint, ymin = -Inf, ymax = Inf),
  #   fill = "#EEE8AA",
  #   color = "#EEE8AA",
  #   alpha = 0.5,
  #   inherit.aes = FALSE
  # ) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    color = guide_legend(override.aes = list(shape = 16, size = 4)),
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  new_scale_fill() +
  # --- NEW: Add a dummy geom_point for the significance legend ---
  # This point won't be plotted (NA coordinates) but its aesthetic will appear in the legend.
  geom_point(
    data = data.frame(fill = "FDR < 0.1 & absolute log2 FC > 1"), # The text for the legend
    aes(fill = fill, x = 0, y = 0),
    shape = '.',
    size = 0,
    inherit.aes = FALSE
  ) +
  scale_fill_manual(
    name = "", # Set the same name for color and fill to merge legends
    values = c(
        "FDR < 0.1 & absolute log2 FC > 1" = "#EEE8AA" # Assign yellow to the new item
    )) +
  # Override aesthetics to ensure legend keys are correct
  guides(
    fill = guide_legend(override.aes = list(shape = 22, size = 5))
  ) +
  geom_vline(xintercept = 0, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = -50, color = 'grey', linetype = 'dotted') +
  geom_vline(xintercept = 50, color = 'grey', linetype = 'dotted') +
  NULL
gene_plot_supplementary_receptors

pdf('spatial_gene_patterning_supplement_receptors.pdf', width = 6, height = 4)
gene_plot_supplementary_receptors
dev.off()

In [ ]:
fig.size(height = 9, width = 6, res = 400)
gene_plot_supplementary_ligands + gene_plot_supplementary_receptors + plot_annotation(tag_levels = 'A') + plot_layout(nrow = 2)
pdf('supplementary_figure_genes_spatial_pattern.pdf', width = 6, height = 9)
gene_plot_supplementary_ligands + gene_plot_supplementary_receptors + plot_annotation(tag_levels = 'A') + plot_layout(nrow = 2)
dev.off()

In [ ]:
states = rownames(.counts)[grepl(rownames(.counts), pattern = 'TGF')]
fig.size(height = 2, width = 9, res = 400)

.temp = .counts[states, ] %>% 
    Matrix::t() %>%
    as.data.frame %>%
    tibble::rownames_to_column('cell_id') %>%
    left_join(., .total_counts)  %>%
    left_join(., .data %>% select(cell_id, dist,closest_interface_type, SampleID)) %>%
    left_join(., tile_metadata %>% select(SampleID, PatientID, Status) %>% distinct) %>%
    na.omit() %>%
    filter(abs(dist) <= 100) %>%
    mutate(Bins = cut(x = dist, breaks = seq(-100, 100, by = 5), include.lowest = TRUE, ordered_result = TRUE)) %>%
    mutate(
        upper_bound = as.numeric(gsub(Bins, pattern = '.*,|)|]', replacement = "")), 
        lower_bound = gsub(.data$Bins, pattern = '\\[|\\(|\\)|,.*', replacement = "") %>% as.numeric,
        midpoint = (upper_bound + lower_bound)/2
    ) %>%
    pivot_longer(cols = all_of(states), names_to = 'Gene', values_to = 'Gene_RNA') %>%
    mutate(.prop = Gene_RNA/total_RNA) 
head(.temp)

.temp %>%
    group_by(Bins, midpoint, PatientID, Status, Gene) %>%
    summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
    mutate(prop_RNA = Gene_RNA/total_RNA) %>%
    ungroup %>%
    mutate(gene_patient = interaction(Gene, PatientID)) %>%
    ggplot() +
        geom_line(aes(x = midpoint, y = prop_RNA, color = Status, group = PatientID), alpha = 0.25, key_glyph = 'point') +
        geom_line(data = .temp %>%
            group_by(Bins, midpoint, PatientID, Status, Gene) %>%
            summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
            mutate(prop_RNA = Gene_RNA/total_RNA) %>%
            ungroup %>%
            group_by(Bins, midpoint, Status, Gene) %>%
            summarize(prop_RNA = median(prop_RNA)) %>%
            ungroup, 
            aes(x = midpoint, y = prop_RNA, color = Status, group = Status), key_glyph = 'point', alpha = 1, 
        ) +
        facet_wrap(~Gene, nrow = 1, scales = 'free_y') +
        cowplot::theme_cowplot(7) +
        scale_y_continuous(labels = scales::percent) + 
        labs(x = 'Microns from interface', y = 'Percent of all transcripts', title = "") +
        theme(legend.position = 'top',
              axis.text.x = element_text(angle = 90, vjust = 1, hjust = 0.5, size = 7)) + # aspect.ratio = 1, 
        guides(color = guide_legend(override.aes = list(shape = 16, size = 7, alpha = 1))) +
        scale_color_manual(values = c("MSI" = "#D55E00","MSS" = "#009E73"), name = 'Interfaces:') +
        NULL

In [ ]:
states = c('CXCL9', 'CXCL10', 'CXCL11', 'CXCR3')
fig.size(height = 2, width = 6, res = 400)

.temp = .counts[states, ] %>% 
    Matrix::t() %>%
    as.data.frame %>%
    tibble::rownames_to_column('cell_id') %>%
    left_join(., .total_counts)  %>%
    left_join(., .data %>% select(cell_id, dist,closest_interface_type, SampleID)) %>%
    left_join(., tile_metadata %>% select(SampleID, PatientID, Status) %>% distinct) %>%
    na.omit() %>%
    filter(abs(dist) <= 100) %>%
    mutate(Bins = cut(x = dist, breaks = seq(-100, 100, by = 5), include.lowest = TRUE, ordered_result = TRUE)) %>%
    mutate(
        upper_bound = as.numeric(gsub(Bins, pattern = '.*,|)|]', replacement = "")), 
        lower_bound = gsub(.data$Bins, pattern = '\\[|\\(|\\)|,.*', replacement = "") %>% as.numeric,
        midpoint = (upper_bound + lower_bound)/2
    ) %>%
    pivot_longer(cols = all_of(states), names_to = 'Gene', values_to = 'Gene_RNA') %>%
    mutate(.prop = Gene_RNA/total_RNA) 
head(.temp)

.temp %>%
    group_by(Bins, midpoint, PatientID, Status, Gene) %>%
    summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
    mutate(prop_RNA = Gene_RNA/total_RNA) %>%
    ungroup %>%
    mutate(gene_patient = interaction(Gene, PatientID)) %>%
    ggplot() +
        geom_line(aes(x = midpoint, y = prop_RNA, color = Status, group = PatientID), alpha = 0.25, key_glyph = 'point') +
        geom_line(data = .temp %>%
            group_by(Bins, midpoint, PatientID, Status, Gene) %>%
            summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
            mutate(prop_RNA = Gene_RNA/total_RNA) %>%
            ungroup %>%
            group_by(Bins, midpoint, Status, Gene) %>%
            summarize(prop_RNA = median(prop_RNA)) %>%
            ungroup, 
            aes(x = midpoint, y = prop_RNA, color = Status, group = Status), key_glyph = 'point', alpha = 1, 
        ) +
        facet_wrap(~Gene, nrow = 1, scales = 'free_y') +
        cowplot::theme_cowplot(7) +
        scale_y_continuous(labels = scales::percent) + 
        labs(x = 'Microns from interface', y = 'Percent of all transcripts', title = "") +
        theme(legend.position = 'top',
              axis.text.x = element_text(angle = 90, vjust = 1, hjust = 0.5, size = 7)) + # aspect.ratio = 1, 
        guides(color = guide_legend(override.aes = list(shape = 16, size = 7, alpha = 1))) +
        scale_color_manual(values = c("MSI" = "#D55E00","MSS" = "#009E73"), name = 'Interfaces:') +
        NULL

In [ ]:
.temp %>%
    group_by(Bins, midpoint, PatientID, Status, Gene) %>%
    summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
    mutate(prop_RNA = Gene_RNA/total_RNA) %>%
    ungroup %>%
    mutate(gene_patient = interaction(Gene, PatientID)) %>%
    ggplot() +
        geom_line(aes(x = midpoint, y = prop_RNA, color = Status, group = PatientID), alpha = 0.25, key_glyph = 'point') +
        geom_line(data = .temp %>%
            group_by(Bins, midpoint, PatientID, Status, Gene) %>%
            summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
            mutate(prop_RNA = Gene_RNA/total_RNA) %>%
            ungroup %>%
            group_by(Bins, midpoint, Status, Gene) %>%
            summarize(prop_RNA = median(prop_RNA)) %>%
            ungroup, 
            aes(x = midpoint, y = prop_RNA, color = Status, group = Status), key_glyph = 'point', alpha = 1, 
        ) +
        facet_wrap(~Gene, nrow = 1, scales = 'free_y') +
        cowplot::theme_cowplot(7) +
        scale_y_continuous(labels = scales::percent) + 
        labs(x = 'Microns from interface', y = 'Percent of all transcripts', title = "") +
        theme(legend.position = 'top',
              axis.text.x = element_text(angle = 90, vjust = 1, hjust = 0.5, size = 7)) + # aspect.ratio = 1, 
        guides(color = guide_legend(override.aes = list(shape = 16, size = 7, alpha = 1))) +
        scale_color_manual(values = c("MSI" = "#D55E00","MSS" = "#009E73"), name = 'Interfaces:') +
        geom_rect(data =  results_combined %>%
mutate("fdr.signif" = p.adj < 0.1 & log2_fold_change > 1) %>%
group_by(Gene) %>%
filter(p.adj < 0.1 & log2_fold_change > 1) %>%
mutate(group = cumsum(fdr.signif != lag(fdr.signif, default = first(fdr.signif)))) %>%
group_by(Gene, group, fdr.signif) %>%
summarise(min_value = min(midpoint),
max_value = max(midpoint),
.groups = 'drop') %>%
select(-group) %>% 
ungroup, #%>%  # & effsize > 0.5
aes(xmin = min_value, xmax = max_value, ymin = -Inf, ymax = Inf), color = '#EEE8AA', fill = '#EEE8AA', inherit.aes = FALSE, alpha = 0.5) + 
NULL


# Show donor-level traces and a median line for multiple genes at the same time: Hub+ vs Hub- interfaces

In [ ]:
.total_counts = data.frame(total_RNA = colSums(.counts)) %>%
    tibble::rownames_to_column('cell_id')

In [ ]:
states = rownames(.counts)[grepl(rownames(.counts), pattern = 'TGF')]
fig.size(height = 2, width = 9, res = 400)

.temp = .counts[states, ] %>% 
    Matrix::t() %>%
    as.data.frame %>%
    tibble::rownames_to_column('cell_id') %>%
    left_join(., .total_counts)  %>%
    left_join(., .data %>% select(cell_id, dist,closest_interface_type, SampleID)) %>%
    left_join(., tile_metadata %>% select(SampleID, PatientID, Status) %>% distinct) %>%
    filter(Status == 'MSI') %>%
    na.omit() %>%
    filter(abs(dist) <= 100) %>%
    mutate(Bins = cut(x = dist, breaks = seq(-100, 100, by = 5), include.lowest = TRUE, ordered_result = TRUE)) %>%
    mutate(
        upper_bound = as.numeric(gsub(Bins, pattern = '.*,|)|]', replacement = "")), 
        lower_bound = gsub(.data$Bins, pattern = '\\[|\\(|\\)|,.*', replacement = "") %>% as.numeric,
        midpoint = (upper_bound + lower_bound)/2
    ) %>%
    pivot_longer(cols = all_of(states), names_to = 'Gene', values_to = 'Gene_RNA') %>%
    mutate(.prop = Gene_RNA/total_RNA) 
head(.temp)

.temp %>%
    group_by(Bins, midpoint, PatientID,closest_interface_type, Gene) %>%
    summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
    mutate(prop_RNA = Gene_RNA/total_RNA) %>%
    ungroup %>%
    mutate(gene_patient = interaction(Gene, PatientID)) %>%
    ggplot() +
        geom_line(aes(x = midpoint, y = prop_RNA, color =closest_interface_type, group = interaction(closest_interface_type, PatientID)), alpha = 0.25, key_glyph = 'point') +
        geom_line(data = .temp %>%
            group_by(Bins, midpoint, PatientID,closest_interface_type, Gene) %>%
            summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
            mutate(prop_RNA = Gene_RNA/total_RNA) %>%
            ungroup %>%
            group_by(Bins, midpoint,closest_interface_type, Gene) %>%
            summarize(prop_RNA = median(prop_RNA)) %>%
            ungroup, 
            aes(x = midpoint, y = prop_RNA, color =closest_interface_type, group =closest_interface_type), key_glyph = 'point', alpha = 1, 
        ) +
        facet_wrap(~Gene, nrow = 1, scales = 'free_y') +
        cowplot::theme_cowplot(7) +
        scale_y_continuous(labels = scales::percent) + 
        labs(x = 'Microns from interface', y = 'Percent of all transcripts', title = "") +
        theme(legend.position = 'top',
              axis.text.x = element_text(angle = 90, vjust = 1, hjust = 0.5, size = 7)) + # aspect.ratio = 1, 
        guides(color = guide_legend(override.aes = list(shape = 16, size = 7, alpha = 1))) +
        scale_color_manual(values = c("Hub+" = "#D55E00","Hub-" = "#009E73"), name = 'Interfaces:') +
        NULL

In [ ]:
states = c('CXCL9', 'CXCL10', 'CXCL11', 'CXCR3')
fig.size(height = 2, width = 6, res = 400)

.temp = .counts[states, ] %>% 
    Matrix::t() %>%
    as.data.frame %>%
    tibble::rownames_to_column('cell_id') %>%
    left_join(., .total_counts)  %>%
    left_join(., .data %>% select(cell_id, dist,closest_interface_type, SampleID)) %>%
    left_join(., tile_metadata %>% select(SampleID, PatientID, Status) %>% distinct) %>%
    filter(Status == 'MSI') %>%
    na.omit() %>%
    filter(abs(dist) <= 100) %>%
    mutate(Bins = cut(x = dist, breaks = seq(-100, 100, by = 5), include.lowest = TRUE, ordered_result = TRUE)) %>%
    mutate(
        upper_bound = as.numeric(gsub(Bins, pattern = '.*,|)|]', replacement = "")), 
        lower_bound = gsub(.data$Bins, pattern = '\\[|\\(|\\)|,.*', replacement = "") %>% as.numeric,
        midpoint = (upper_bound + lower_bound)/2
    ) %>%
    pivot_longer(cols = all_of(states), names_to = 'Gene', values_to = 'Gene_RNA') %>%
    mutate(.prop = Gene_RNA/total_RNA) 
head(.temp)

.temp %>%
    group_by(Bins, midpoint, PatientID,closest_interface_type, Gene) %>%
    summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
    mutate(prop_RNA = Gene_RNA/total_RNA) %>%
    ungroup %>%
    mutate(gene_patient = interaction(Gene, PatientID)) %>%
    ggplot() +
        geom_line(aes(x = midpoint, y = prop_RNA, color =closest_interface_type, group = interaction(closest_interface_type, PatientID)), alpha = 0.25, key_glyph = 'point') +
        geom_line(data = .temp %>%
            group_by(Bins, midpoint, PatientID,closest_interface_type, Gene) %>%
            summarize(total_RNA = sum(total_RNA), Gene_RNA = sum(Gene_RNA)) %>%
            mutate(prop_RNA = Gene_RNA/total_RNA) %>%
            ungroup %>%
            group_by(Bins, midpoint,closest_interface_type, Gene) %>%
            summarize(prop_RNA = median(prop_RNA)) %>%
            ungroup, 
            aes(x = midpoint, y = prop_RNA, color =closest_interface_type, group =closest_interface_type), key_glyph = 'point', alpha = 1, 
        ) +
        facet_wrap(~Gene, nrow = 1, scales = 'free_y') +
        cowplot::theme_cowplot(7) +
        scale_y_continuous(labels = scales::percent) + 
        labs(x = 'Microns from interface', y = 'Percent of all transcripts', title = "") +
        theme(legend.position = 'top',
              axis.text.x = element_text(angle = 90, vjust = 1, hjust = 0.5, size = 7)) + # aspect.ratio = 1, 
        guides(color = guide_legend(override.aes = list(shape = 16, size = 7, alpha = 1))) +
        scale_color_manual(values = c("Hub+" = "#D55E00","Hub-" = "#009E73"), name = 'Interfaces:') +
        NULL

In [ ]:
# Install necessary packages if you haven't already
# install.packages("dplyr")
# install.packages("rstatix")
# install.packages("tibble")

library(dplyr)
library(rstatix)
library(tibble)

# --- 1. Create a Sample Dataframe with a 'Gene' column ---
# This simulates your data structure with multiple genes.
# Each patient is still in only oneclosest_interface_type.
set.seed(42) # for reproducible results
df <- tibble(
  Gene = rep(c("GeneX", "GeneY"), each = 60),
  midpoint = rep(rep(c(10, 20, 30), each = 20), 2),
  prop_RNA = c(
    # Data for GeneX
    rnorm(20, 0.5, 0.1), rnorm(20, 0.7, 0.15), rnorm(20, 0.4, 0.08),
    # Data for GeneY
    rnorm(20, 0.8, 0.1), rnorm(20, 0.6, 0.15), rnorm(20, 0.9, 0.08)
  ),
  PatientID = paste0("Patient_", 1:120),
 closest_interface_type = rep(c(rep("Type_A", 10), rep("Type_B", 10)), 6)
)

# --- 2. Check Patient Representation ---
# This check confirms that all unique patients from the original dataframe are
# being considered in the analysis.
total_unique_patients <- df %>% distinct(PatientID) %>% nrow()
cat(paste("Total unique patients in the original dataframe:", total_unique_patients, "\n\n"))


# --- 3. Perform Grouped Wilcoxon Test for Each Gene and Adjust P-values ---
# The pipeline will:
#  - Group the data by each 'Gene' AND each unique 'midpoint'.
#  - Perform a Wilcoxon test comparing 'prop_RNA' between the two 'interface_type' groups.
#  - Ungroup the data so p-value adjustment is performed across ALL tests.
#  - Add an adjusted p-value (p.adj) to account for multiple testing.
#    The "BH" (Benjamini-Hochberg) method is a common choice for FDR control.
results <- df %>%
  group_by(Gene, midpoint) %>%
  wilcox_test(prop_RNA ~closest_interface_type, detailed = TRUE) %>%
  ungroup() %>% # Ungroup to adjust p-values across all genes and midpoints
  adjust_pvalue(method = "BH") %>%
  add_significance("p.adj")

# The `detailed = TRUE` argument adds confidence intervals and the estimate.

# --- 4. Display the Results ---
cat("Wilcoxon Test Results (with adjusted p-values):\n")
print(results)


# --- 5. Verify Patient Counts in the Analysis ---
# The 'n1' and 'n2' columns in the results table show the number of samples
# (patients) used in each comparison for Type_A and Type_B, respectively.
# This confirms how many patients were included in each individual test.
patient_counts_in_analysis <- results %>%
    select(Gene, midpoint, n1, n2) %>%
    mutate(total_patients_in_test = n1 + n2)

cat("\nPatient counts used in each test:\n")
print(patient_counts_in_analysis)



In [ ]:
sessionInfo()